In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

spark =  SparkSession.builder \
    .master('local[*]')\
    .appName('test') \
    .getOrCreate()

24/03/02 09:42:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [5]:
!wc -l fhv_tripdata_2019-10.csv.gz

62958 fhv_tripdata_2019-10.csv.gz


In [6]:
df = spark.read\
    .option('header','true') \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [7]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropOff_datetime,StringType,true),StructField(PUlocationID,StringType,true),StructField(DOlocationID,StringType,true),StructField(SR_Flag,StringType,true),StructField(Affiliated_base_number,StringType,true)))

In [8]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
+--------------------+------------------

In [9]:
# %%timeit
import pandas as pd
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz')

In [10]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [11]:
df_pandas['Affiliated_base_number'] = df_pandas['Affiliated_base_number'].fillna("NAN Values")

In [12]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropOff_datetime,StringType,true),StructField(PUlocationID,DoubleType,true),StructField(DOlocationID,DoubleType,true),StructField(SR_Flag,DoubleType,true),StructField(Affiliated_base_number,StringType,true)))

Define the new shema and read the data again by pasisng in the data types

In [28]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropOff_datetime',types.TimestampType(),True),
    types.StructField('PUlocationID',types.IntegerType(),True),
    types.StructField('DOlocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True),
    types.StructField('Affiliated_base_number',types.StringType(),True)
])

df = spark.read \
    .option('header', 'true') \
    .schema(schema)\
    .csv('fhv_tripdata_2019-10.csv.gz')

In [29]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropOff_datetime,TimestampType,true),StructField(PUlocationID,IntegerType,true),StructField(DOlocationID,IntegerType,true),StructField(SR_Flag,StringType,true),StructField(Affiliated_base_number,StringType,true)))

In [30]:
df = df.repartition(24)

In [31]:
df.write.parquet('fhvhv/2019/01')